<h1>Tattooist</h1> <br>
Tattooist as an overall product is a reservation system for tattoo artists. This part of the project focuses on one of the key features - recommendation system. 

Tattoo artists upload examples of their work. These photos are then used as training data for image classification model which serves as a recommendation system for customers. A user uploads a picture of their desired tattoo style from the Internet. Based on this photo, said classification model recommends suitable tattoo artists in the area.

<i>Still in progress. Not publicly available in production yet.</i>

The current workflow is that the classification model is converted to a CoreML model which is meant to sit in cloud and be downloaded each time a user opens an app (providing a new version of the model is available). It is to be yet discussed whether sending API requests to a model in cloud would not be the more efficient way.

In [1]:
import cv2
import os
import numpy as np
import boto3
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Concatenate
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import coremltools as ct

scikit-learn version 1.3.0 is not supported. Minimum required version: 0.17. Maximum required version: 1.1.2. Disabling scikit-learn conversion API.
TensorFlow version 2.15.0 has not been tested with coremltools. You may run into unexpected errors. TensorFlow 2.12.0 is the most recent version that has been tested.


In [19]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    rescale=1./255) 

# Photos directory (currently local, in full production in AWS S3)
train_dir = 'DataV3/Train'
validation_dir = 'DataV3/Valid'

# Load images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse')

Found 478 images belonging to 10 classes.
Found 48 images belonging to 10 classes.


In [18]:
def count_photos_in_subfolders(directory):
    for subdir, dirs, files in os.walk(directory):
        if subdir == directory:
            continue
        photo_count = len(files)
        subfolder_name = os.path.basename(subdir)
        print(f"Number of photos in {subfolder_name}: {photo_count}")

print("Train Directory:")
count_photos_in_subfolders('DataV3/Train')

print("\nValidation Directory:")
count_photos_in_subfolders('DataV3/Valid')

Train Directory:
Number of photos in dariastahp: 36
Number of photos in bod.yx: 30
Number of photos in maison_hefner: 29
Number of photos in rotopet: 54
Number of photos in gabi.tetu: 63
Number of photos in sunshine_ink: 65
Number of photos in obrazkynatelo: 31
Number of photos in bronislava_orlicka: 37
Number of photos in daf647: 73
Number of photos in duhovka.ink: 70

Validation Directory:
Number of photos in dariastahp: 5
Number of photos in bod.yx: 5
Number of photos in maison_hefner: 3
Number of photos in rotopet: 5
Number of photos in gabi.tetu: 5
Number of photos in sunshine_ink: 5
Number of photos in obrazkynatelo: 5
Number of photos in bronislava_orlicka: 5
Number of photos in daf647: 5
Number of photos in duhovka.ink: 5


In [20]:
total_validation_samples = len(validation_generator.filenames)
batch_size = validation_generator.batch_size
validation_steps = total_validation_samples // batch_size

In [21]:
input_tensor = Input(shape=(224, 224, 3))
base_model = DenseNet121(include_top=False, input_shape=(224, 224, 3), weights='imagenet', input_tensor=input_tensor)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(units=10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
total_training_images = 478
batch_size = 32
steps_per_epoch = total_training_images / batch_size

In [24]:
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch, 
    epochs=15,
    validation_data=validation_generator,
    validation_steps=validation_steps) 

Epoch 1/15
14/14 [==============================] - 21s 1s/step - loss: 2.9943 - accuracy: 0.1360 - val_loss: 2.5647 - val_accuracy: 0.1250
Epoch 2/15
14/14 [==============================] - 15s 953ms/step - loss: 2.5642 - accuracy: 0.1778 - val_loss: 2.2472 - val_accuracy: 0.2188
Epoch 3/15
14/14 [==============================] - 14s 907ms/step - loss: 2.2681 - accuracy: 0.2469 - val_loss: 1.8766 - val_accuracy: 0.3125
Epoch 4/15
14/14 [==============================] - 14s 931ms/step - loss: 2.1268 - accuracy: 0.3285 - val_loss: 1.7243 - val_accuracy: 0.4375
Epoch 5/15
14/14 [==============================] - 14s 935ms/step - loss: 1.8963 - accuracy: 0.3577 - val_loss: 1.5681 - val_accuracy: 0.5000
Epoch 6/15
14/14 [==============================] - 14s 933ms/step - loss: 1.6570 - accuracy: 0.4582 - val_loss: 1.3111 - val_accuracy: 0.6875
Epoch 7/15
14/14 [==============================] - 14s 917ms/step - loss: 1.5811 - accuracy: 0.4791 - val_loss: 1.3078 - val_accuracy: 0.6875
Ep

In [26]:
#Saving Keras model

model.save('Keras/')

In [27]:
# Getting class names for the CoreML model

class_indices = train_generator.class_indices
class_names = sorted(class_indices, key=class_indices.get)  # Sort class names by their indices
print(class_names)

# Converting model to CoreML model suitable for iOS applications

mlmodel = ct.convert(model, inputs=[ct.ImageType(scale=1/255.0)], 
                    classifier_config=ct.ClassifierConfig(class_names),
                     convert_to="neuralnetwork")

Running TensorFlow Graph Passes: 100%|███████| 6/6 [00:00<00:00, 17.76 passes/s]
Converting TF Frontend ==> MIL Ops: 100%|█| 1099/1099 [00:00<00:00, 4801.35 ops/
Running MIL frontend_tensorflow2 pipeline: 100%|█| 7/7 [00:00<00:00, 344.98 pass
Running MIL default pipeline: 100%|████████| 69/69 [00:01<00:00, 45.81 passes/s]
Running MIL backend_neuralnetwork pipeline: 100%|█| 9/9 [00:00<00:00, 477.90 pas
Translating MIL ==> NeuralNetwork Ops: 100%|█| 1611/1611 [00:02<00:00, 670.51 op


In [29]:
# Saving CoreML model

ct.models.utils.save_spec(mlmodel.get_spec(), 'CoreML.mlpackage')